# Backpropogation from Scratch 

#### Final Project for COMPSCIX404 - Data Structures and Algorithms
#### Nikhil Gopal 
#### August 2024

## Introduction

In this project, I will build the backpropogation algorithm, and show how it can be used to compute the gradient of the cost function when training a Neural Network. Neural Networks trained with backpropogation can be applied to text generation, computer vision, classification, facial recognition, audio generation, speech to text and many more applications.

I will start by explaining the algorithm and it's applicability to modern Deep Learning, with an overview of the mathematics behind it. We will discuss how Dynamic Programming techniques are leveraged to make the algorithm more efficient. Finally, I will analyze the time and space complexity of the algorithm before presenting a code implementation.

Finally, we'll use the Pytorch Deep Learning Framework to show a real life example of how a Data Scientist or Deep Learning Engineer would train a Neural Network to classify handwritten digits from the popular MNSIT dataset. Pytorch automatically computes derivatives and gradients for you with the `loss.backwards()` method, but I will still show it so that you get an understanding of how this would be applied in the real world. 




## Motivation

Fundamentally, every problem in Machine Learning is an **optimization problem**. Optimization is the process of finding the maximum or minimum value of a function, such that it behaves optimally. In the case of Machine Learning, we seek to optimize the *loss function*, which describes the difference between the expected output of an ML model, and it's actual output. For example, if we were predicting human height and our model predicted 100 cm for a person who's true height was 105 cm, the loss would be $105-100=5$ cm. ML models have changable *parameters* whose optimal values can be learned from data such that the model's loss function is as close as possible to zero. 

Deep Learning is a subsect of Machine Learning that relies on a specific model called a Neural Network. Neural Networks pass inputs through different "layers" of the model, which all contribute to the model's final output. These models often have much higher numbers of learnable parameters than simpler models, but the computations corresponding to a single input must be performed sequentially and cannot be parallelized, since the input of a later layer must be the output of a previous layer. Activation functions are applied to force a layer's output to fall within a specified range of values, to fall within a set of values, or to generate a probability distribution of values:


![title](/Users/nikhilgopal/data-structures-algos/imgs/nn-img.png)

Many advancements in Machine Learning have come from using Deep Neural Networks (models with lots of layers and parameters) to learn the relevant features of and transform inputted data in novel ways. By leveraging techniques from Calculus, we can compute the gradient vector of a Neural Network's cost function. Each index of the gradient vector contains the partial derivative of the cost function with respect to a parameter, which describes the impact that each individual parameter has on the total loss of the network. The sum of all of the loss that comes from every input parameter is the total loss of the network, thus making computing the gradient a key factor in minimizing the loss function.

Once the gradient is computed, the model's paramaters are adjusted using a parameter estimation method such as gradient descent, and the loss and parameters are continously adjusted and recomputed until the model's behavior more closely mimics the information it learned from the training data.

As model sizes keep increasing, an efficient algorithm is needed to compute the gradient. This algorithm should ideally be computationally efficient so as to minimize total training time and memory usage when training neural networks. This will ensure that models can be deployed into producition faster, and that they can be made more accuracte by being able to be trained on more data in less time.



## Introduction to backpropagation

To summarize, we need to compute:

$\text{Loss} = L(x) = \sum_{i=1}^{n} (y_i - \hat{y}_i)$

Where $y_i$ is the true value for a given observation and $\hat{y}_i$ is the value predicted by the model.

We also know that the loss function is parameterized by the weights and biases of the network, denoted as $\theta$:

 $\text{Loss} = L(x; \theta) = \sum_{i=1}^{n} (y_i - \hat{y}_i)$

To compute this loss, we must compute the gradient vector $\nabla L(x)$:

$\nabla L = \left( \frac{\partial L}{\partial w_1}, \frac{\partial L}{\partial w_2}, \ldots, \frac{\partial L}{\partial w_n}, \frac{\partial L}{\partial b_1}, \frac{\partial L}{\partial b_2}, \ldots, \frac{\partial L}{\partial b_m} \right)$

$\nabla L(x)$ contains the partial derivatives of the weights and bias in the network. The different elements of this vector tell us the impact that changing each parameter by a unit of 1 will have on the total loss.

Finally, using the partial derivatives we'll update the parameters using gradient descent: 
$w_i \leftarrow w_i - \eta \frac{\partial L}{\partial w_i}$ where $\eta$ is the learning rate of our network

By leveraging some clever math explained [here](http://neuralnetworksanddeeplearning.com/chap2.html), we arrive at these fundamental equations of backpropogation, which explain how to calculate the necessary partial derivatives:

![title](imgs/backprop-equations.png)

Equation (BP1):

 $\delta^L = \nabla_a C \odot \sigma{\prime}(z^L)$

-	$\delta^L$: This represents the error in the output layer L.
-	$\nabla_a C$: This is the gradient of the cost function C with respect to the activations a in the output layer. It measures how the cost changes with the output activations.
-	$\odot$: This denotes the Hadamard product (element-wise multiplication).
-	$\sigma{\prime}(z^L)$: This is the derivative of the activation function $\sigma$ with respect to the input $z^L$ to the output layer.

Explanation: This equation calculates the error at the output layer by combining the gradient of the cost function and the derivative of the activation function.

Equation (BP2):

$\delta^l = ((w^{l+1})^T \delta^{l+1}) \odot \sigma{\prime}(z^l)$

-	$\delta^l$: This represents the error in layer $l$.
-	$w^{l+1}$: This is the weight matrix connecting layer $l$ to layer $l+1$.
-	$(w^{l+1})^T$: This is the transpose of the weight matrix $w^{l+1}$.
-	$\delta^{l+1}$: This is the error in the next layer, $l+1$.
-	$\sigma{\prime}(z^l)$: This is the derivative of the activation function with respect to the input $z^l$ to layer $l$.

Explanation: This equation calculates the error for layer $l$ by propagating the error from the next layer $l+1$ backwards through the network, adjusting for the activation function’s derivative.

Equation (BP3):

$\frac{\partial C}{\partial b_j^l} = \delta_j^l$ 

-	$\frac{\partial C}{\partial b_j^l}$: This is the partial derivative of the cost function $C$ with respect to the bias $b_j^l$ in layer $l$.
-	$\delta_j^l$: This is the error for neuron $j$ in layer $l$.

Explanation: This equation states that the gradient of the cost function with respect to the bias is simply the error term $\delta_j^l$.

Equation (BP4):

$\frac{\partial C}{\partial w_{jk}^l} = a_k^{l-1} \delta_j^l$ 

-	$\frac{\partial C}{\partial w_{jk}^l}$: This is the partial derivative of the cost function C with respect to the weight $w_{jk}^l$ connecting neuron $k$ in layer $l-1$ to neuron $j$ in layer $l$.
-	$a_k^{l-1}$: This is the activation of neuron $k$ in the previous layer $l-1$.
-	$\delta_j^l$: This is the error for neuron $j$ in layer $l$.

Explanation: This equation calculates the gradient of the cost function with respect to the weights. It shows that this gradient is the product of the error term $\delta_j^l$ and the activation of the neuron in the previous layer $a_k^{l-1}$.

## Pseudocode

Having seen these equations, your data structures and algorithms knowledge should be sounding alarms in your brain that calculations can be reused to optimize the efficiency of our gradient calculuations!

* Before any loss is calculated $W^T$ is initialized during the forward pass to calculate the prediction. This calculation should be saved and not redone during the loss calculation
* In the second equation, we observe that the part of the computation for the loss of a given layer includes $\sigma{\prime}(z^l)$, which represents the loss of the next layer in the network. By starting at the end of the network and applying the chain rule to calculate the gradient backwards, we can reduce the total number of computations by reusing derivatives from previous layers! 
  * The above applies to individual neurons and not just layers as well (BP4)
* By leveraging matrix multiplications, we can reduce runtime (not time complexity). The overall number of computations will be the same, but since multiplications in matrix math do not need to be performed sequentially, we can leverage GPUs and parallel processing to compute them in parallel and reduce overall algorithm runtime. 

Algorithm Pseudo code:

```python
def forward_pass(weights, biases):
  activation_funcation(weight + bias)

def backprop(weights):
  
  gradient = []

  for x in range(output_layer:input_layer):  
    if layer == final:
      dloss = derivative(layer)
      gradient[current_layer] = dloss
    else:
      dloss = derivative(current_layer)*gradient[layer+1]
      gradient[current_layer] = dloss

  return gradient
```

1. Initialize Gradient Arrays

	-	Create zero-filled arrays for the gradients of the biases ($nabla_b$) and weights ($nabla_w$).
	-	These arrays should match the shapes of the biases and weights in your network.

2. Forward Pass

	-	Start with the input activation ($a = x$).
	-	For each layer in the network:
	-	Compute the weighted input $z$ as the dot product of the weights and the activation from the previous layer, plus the bias.
	-	Apply the activation function (e.g., sigmoid) to $z$ to get the new activation.
	-	Store both $z$ and the new activation for use in the backward pass.

3. Backward Pass for Output Layer

	-	Compute the error term ($\delta_L$) for the output layer:
	-	Calculate the derivative of the cost function with respect to the output activations.
	-	Multiply this by the derivative of the activation function evaluated at the weighted input $z$ of the output layer.
	-	Update the gradients for the output layer’s biases and weights using this error term.

4. Backward Pass for Hidden Layers

	-	For each hidden layer, moving backward:
	-	Compute the error term ($\delta$) for the current layer:
	-	Multiply the transpose of the weights from the next layer by the error term from the next layer.
	-	Multiply element-wise by the derivative of the activation function evaluated at the weighted input $z$ of the current layer.
	-	Update the gradients for the current layer’s biases and weights using this error term.

5. Calculate Gradients

	-	For each layer:
	-	The gradient for the biases is the error term ($\delta$) itself.
	-	The gradient for the weights is the dot product of the error term ($\delta$) and the transpose of the activations from the previous layer.

6. Return Gradients

	-	Finally, return gradients

## Code

Credit to Michael Nielsen who wrote the code for this Neural Network class implementation in this tutorial: [link](https://neuralnetworksanddeeplearning.com/chap1.html#implementing_our_network_to_classify_digits)

Since this assignment is focused specifically on backpropogation, I will write my own backpropogation implementation. To save time, the rest of the code I took from his git repo for the Python 3 implementation ([link](https://github.com/unexploredtest/neural-networks-and-deep-learning.git))


In [1]:
#! git clone https://github.com/unexploredtest/neural-networks-and-deep-learning.git

In [56]:
import sys
sys.path.append('neural-networks-and-deep-learning/src')

import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [57]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
import random
import time
import numpy as np


class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            time1 = time.time()
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            time2 = time.time()
            # if test_data:
            #     print("Epoch {0}: {1} {2}, took {3:.2f} seconds".format(
            #         j, self.evaluate(test_data), n_test, time2-time1))
            # else:
            #     print("Epoch {0} complete in {1:.2f} seconds".format(j, time2-time1))
            if test_data:
                correct = self.evaluate(test_data)
                accuracy = (correct / n_test) * 100
                print("Epoch {}, predicted correctly: {:.2f}%".format(
                    j+1, accuracy))
            else:
                print("Epoch {} complete".format(j+1))
                
    

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [58]:
class NikhilNet(Network):

    def backprop(self, x, y):
        """Return the gradient of the cost function. X represents the input to the network and Y represents the data's labels."""
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Forward pass
        activation = x
        activations = [x]
        weighted_inputs = []
        activation_derivatives = []
        activation_transposes = [x.T]  
        
        #compute z (weighted inputs to layers), activations, activation derivatives and matrix transposes to reuse later
        for layer in range(len(self.weights)):
            weighted_input = np.dot(self.weights[layer], activation) + self.biases[layer]
            weighted_inputs.append(weighted_input)
            activation = sigmoid(weighted_input)
            activations.append(activation)
            activation_derivatives.append(sigmoid_prime(weighted_input))
            activation_transposes.append(activation.T)  # Store transpose
        
        # Backward pass
        loss = self.cost_derivative(activations[-1], y) * activation_derivatives[-1]
        
        #print(f"Output layer loss = cost_derivative * sigmoid_prime(weighted_input)")
        
        
        nabla_b[-1] = loss
        nabla_w[-1] = np.dot(loss, activation_transposes[-2])
        
        #backpropogate loss through network
        for layer in range(2, self.num_layers):
            loss = np.dot(self.weights[-layer+1].T, loss) * activation_derivatives[-layer]
            
            #print(f"\nLayer {self.num_layers - layer} loss = (weights_{self.num_layers - layer + 1}.T * loss_{self.num_layers - layer + 1}) * sigmoid_prime(weighted_input_{self.num_layers - layer})")
            
            
            nabla_b[-layer] = loss
            nabla_w[-layer] = np.dot(loss, activation_transposes[-layer-1])
        
        return (nabla_b, nabla_w)

net = NikhilNet([784, 30, 10])

net.SGD(training_data, 5, 10, 3.0, test_data=test_data)

Epoch 1, predicted correctly: 81.63%
Epoch 2, predicted correctly: 83.09%
Epoch 3, predicted correctly: 83.32%
Epoch 4, predicted correctly: 84.45%
Epoch 5, predicted correctly: 84.48%


# Complexity Analysis

Now, let's analyze the time and space complexity backpropogation algorithm. We'll break it down, section by section:

```python
def backprop(self, x, y):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # Forward pass
        activation = x
        activations = [x]
        weighted_inputs = []
        activation_derivatives = []
        activation_transposes = [x.T]  
        
        #compute z (weighted inputs to layers), activations, activation derivatives and matrix transposes to reuse later
        for layer in range(len(self.weights)):
            weighted_input = np.dot(self.weights[layer], activation) + self.biases[layer]
            weighted_inputs.append(weighted_input)
            activation = sigmoid(weighted_input)
            activations.append(activation)
            activation_derivatives.append(sigmoid_prime(weighted_input))
            activation_transposes.append(activation.T) 
```

First, we do a forward pass through the network to initialize the weights and biases, and create two arrays that we can use to hold them. We'll also create arrays to store activations, weighted inputs to neurons, derivatives of the activations, and the transposes of the activations. During the forward pass through the network, all of these things can be calculated and saved for later, since they can be reused. 

Note that **Dynamic Programming** techniques are in use here. We are using a **bottom-up approach** to pass through the network, breaking the calculation of each layer into a subproblem, and solving them sequentially instead of trying to solve the entire problem at the same time. We are also employing **memoization**, by storing values to reuse later such as activations, derivatives and weighted inputs. Note that this involves a tradeoff between time and space. Computing all of these things on demand would rquire more overall computations (not significant here enough to change our final Big O). By caching these values in arrays, we can retrieve them in $O(1)$ time at the cost of some extra space. Since I know I am implementing this algorithm to use with small network sizes, using less space is appropriate.






```python

# Backward pass
        loss = self.cost_derivative(activations[-1], y) * activation_derivatives[-1]
        
        #print(f"Output layer loss = cost_derivative * sigmoid_prime(weighted_input)")
        
        
        nabla_b[-1] = loss
        nabla_w[-1] = np.dot(loss, activation_transposes[-2])
        
        #backpropogate loss through network
        for layer in range(2, self.num_layers):
            loss = np.dot(self.weights[-layer+1].T, loss) * activation_derivatives[-layer]
            
            #print(f"\nLayer {self.num_layers - layer} loss = (weights_{self.num_layers - layer + 1}.T * loss_{self.num_layers - layer + 1}) * sigmoid_prime(weighted_input_{self.num_layers - layer})")
            
            
            nabla_b[-layer] = loss
            nabla_w[-layer] = np.dot(loss, activation_transposes[-layer-1])
        
        return (nabla_b, nabla_w)

```

## Practical Example

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

At this step, we define transformations that will turn our image datasets into tensors that pytorch can use, and normalize them, with mean $0.5$ and standard deviation $0.5$, so that values are approximately between the range $[-1,1]$. This is optimal for the allowing the gradient descent algorithm to converge to a true local minimum, and would be very necessary if you were using input features with different scales (though we aren't here).

Next, we download training and test datasets, and apply our transformation onto them, and define our batch size. This helps improve computational efficiency by computing the gradient and loss of a batch of weights, and then adjusting the parameters based on a batch, instead of once for each image.

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Next we'll define a Neural Network class, as a child class of PyTorch's nn.Module class. In the initialization, we'll first use the initialization of the parent function, and then create our layers. Each MNIST image is a 28x28 image. After applying the transformation, pytorch flattens this automatically  into a 1D 28*28 = 784 length vector in the input layer. 

The first hidden layer fc1 thus takes 28*28 input nodes, and outputs 128, which is the input size of the next hidden layer. We then reduce the dimensionality to 64 before passing through the second activation layer. Finally, we have 10 possible outputs, representing one of the ten possible digits. 

In [4]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

        
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


Next, let's create an instance of the neural net class, and then define our loss function and parameter estimation method. Cross entropy loss is a common loss function for categorical data, and stochastic gradient descent is a commonly used algorithm to update weights.

In [5]:
model = NeuralNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

Now we'll train our model over 5 epochs. Notice how the loss decreases with each iteration. This is the network learning in action. The model is using the computed gradient to calculate loss, and then adjust the weights using stochastic gradient descent!

In [6]:
for epoch in range(5):  # Num of epochs
    running_loss = 0.0
    for inputs, labels in trainloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')

Epoch 1, Loss: 0.4260704751565322
Epoch 2, Loss: 0.17622237244664607
Epoch 3, Loss: 0.12769272889314429
Epoch 4, Loss: 0.10216637423484405
Epoch 5, Loss: 0.0845510232399331
Finished Training


Finally, we test our model's accuracy on the test set:

In [7]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total} %')

Accuracy on the test set: 96.68 %
